In [19]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 30 20:11:19 2016

@author: stephen
"""
 
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 2000
folder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist  = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr(folder+fname+'/'+fname+'_TRAIN')
    x_test, y_test = readucr(folder+fname+'/'+fname+'_TEST')
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0]/10, 16)
    
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))

    x = keras.layers.Input(x_train.shape[1:])
#    drop_out = Dropout(0.2)(x)
    conv1 = keras.layers.Conv2D(128, 8, 1, border_mode='same')(x)
    conv1 = keras.layers.normalization.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation('relu')(conv1)
    
#    drop_out = Dropout(0.2)(conv1)
    conv2 = keras.layers.Conv2D(256, 5, 1, border_mode='same')(conv1)
    conv2 = keras.layers.normalization.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    
#    drop_out = Dropout(0.2)(conv2)
    conv3 = keras.layers.Conv2D(128, 3, 1, border_mode='same')(conv2)
    conv3 = keras.layers.normalization.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    
    full = keras.layers.pooling.GlobalAveragePooling2D()(conv3)    
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    
    
    model = Model(input=x, output=out)
     
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    
    
    
    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    
    
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (8, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 390 samples, validate on 391 samples
Epoch 1/2000
390/390 [==============================] - 9s - loss: 3.6603 - acc: 0.0179 - val_loss: 3.6139 - val_acc: 0.0537
Epoch 2/2000
390/390 [==============================] - 5s - loss: 3.4711 - acc: 0.0897 - val_loss: 3.6554 - val_acc: 0.0281
Epoch 3/2000
390/390 [==============================] - 5s - loss: 3.3238 - acc: 0.1154 - val_loss: 3.8639 - val_acc: 0.0281
Epoch 4/2000
390/390 [==============================] - 5s - loss: 3.1978 - acc: 0.1564 - val_loss: 5.0258 - val_acc: 0.0281
Epoch 5/2000
390/390 [==============================] - 5s - loss: 3.0789 - acc: 0.1590 - val_loss: 7.4311 - val_acc: 0.0281
Epoch 6/2000
390/390 [==============================] - 5s - loss: 2.9847 - acc: 0.2205 - val_loss: 10.2021 - val_acc: 0.0281
Epoch 7/2000
390/390 [==============================] - 5s - loss: 2.9049 - acc: 0.2282 - val_loss: 12.8319 - val_acc: 0.0281
Epoch 8/2000
390/390 [==============================] - 5s - loss: 2.8314 - a

390/390 [==============================] - 5s - loss: 1.2204 - acc: 0.6769 - val_loss: 2.4747 - val_acc: 0.2353
Epoch 66/2000
390/390 [==============================] - 5s - loss: 1.1686 - acc: 0.6923 - val_loss: 1.7831 - val_acc: 0.4348
Epoch 67/2000
390/390 [==============================] - 5s - loss: 1.2160 - acc: 0.6564 - val_loss: 4.8560 - val_acc: 0.1893
Epoch 68/2000
390/390 [==============================] - 5s - loss: 1.2419 - acc: 0.6359 - val_loss: 2.6075 - val_acc: 0.2660
Epoch 69/2000
390/390 [==============================] - 5s - loss: 1.2175 - acc: 0.6513 - val_loss: 1.7158 - val_acc: 0.4271
Epoch 70/2000
390/390 [==============================] - 5s - loss: 1.3496 - acc: 0.6231 - val_loss: 4.1016 - val_acc: 0.1816
Epoch 71/2000
390/390 [==============================] - 5s - loss: 1.1354 - acc: 0.7179 - val_loss: 1.2967 - val_acc: 0.6368
Epoch 72/2000
390/390 [==============================] - 5s - loss: 1.1643 - acc: 0.6718 - val_loss: 1.4091 - val_acc: 0.5396
Epoch 

390/390 [==============================] - 5s - loss: 0.9378 - acc: 0.7179 - val_loss: 2.8454 - val_acc: 0.3043
Epoch 130/2000
390/390 [==============================] - 5s - loss: 0.8082 - acc: 0.7641 - val_loss: 2.6459 - val_acc: 0.3120
Epoch 131/2000
390/390 [==============================] - 5s - loss: 0.8095 - acc: 0.7513 - val_loss: 1.9291 - val_acc: 0.4220
Epoch 132/2000
390/390 [==============================] - 5s - loss: 0.7957 - acc: 0.7385 - val_loss: 1.2401 - val_acc: 0.5959
Epoch 133/2000
390/390 [==============================] - 5s - loss: 0.7236 - acc: 0.7538 - val_loss: 2.6092 - val_acc: 0.3529
Epoch 134/2000
390/390 [==============================] - 5s - loss: 0.7959 - acc: 0.7436 - val_loss: 1.7618 - val_acc: 0.4962
Epoch 135/2000
390/390 [==============================] - 5s - loss: 0.7513 - acc: 0.7846 - val_loss: 1.7423 - val_acc: 0.3964
Epoch 136/2000
390/390 [==============================] - 5s - loss: 0.7275 - acc: 0.7513 - val_loss: 2.9143 - val_acc: 0.3248

390/390 [==============================] - 5s - loss: 0.5449 - acc: 0.8513 - val_loss: 0.8184 - val_acc: 0.7289
Epoch 194/2000
390/390 [==============================] - 5s - loss: 0.6266 - acc: 0.7897 - val_loss: 1.7552 - val_acc: 0.43220.784 - ETA: 2s - loss: 0.6676 
Epoch 195/2000
390/390 [==============================] - 5s - loss: 0.6231 - acc: 0.8179 - val_loss: 0.7994 - val_acc: 0.7366
Epoch 196/2000
390/390 [==============================] - 5s - loss: 0.5933 - acc: 0.7949 - val_loss: 1.1141 - val_acc: 0.5985
Epoch 197/2000
390/390 [==============================] - 5s - loss: 0.5613 - acc: 0.8308 - val_loss: 1.6452 - val_acc: 0.5192
Epoch 198/2000
390/390 [==============================] - 5s - loss: 0.5473 - acc: 0.8436 - val_loss: 1.6891 - val_acc: 0.4578
Epoch 199/2000
390/390 [==============================] - 5s - loss: 0.5656 - acc: 0.8256 - val_loss: 2.9859 - val_acc: 0.3171
Epoch 200/2000
390/390 [==============================] - 5s - loss: 0.5402 - acc: 0.8282 - val

...

390/390 [==============================] - 6s - loss: 0.0299 - acc: 0.9974 - val_loss: 0.5185 - val_acc: 0.8440
Epoch 1729/2000
390/390 [==============================] - 6s - loss: 0.0246 - acc: 0.9949 - val_loss: 0.5101 - val_acc: 0.8465
Epoch 1730/2000
390/390 [==============================] - 6s - loss: 0.0270 - acc: 0.9974 - val_loss: 0.5370 - val_acc: 0.8389
Epoch 1731/2000
390/390 [==============================] - 6s - loss: 0.0296 - acc: 1.0000 - val_loss: 0.5259 - val_acc: 0.8465
Epoch 1732/2000
390/390 [==============================] - 6s - loss: 0.0285 - acc: 0.9974 - val_loss: 0.5086 - val_acc: 0.8440
Epoch 1733/2000
390/390 [==============================] - 6s - loss: 0.0281 - acc: 1.0000 - val_loss: 0.5127 - val_acc: 0.8517
Epoch 1734/2000
390/390 [==============================] - 6s - loss: 0.0320 - acc: 0.9949 - val_loss: 0.5080 - val_acc: 0.8414
Epoch 1735/2000
390/390 [==============================] - 6s - loss: 0.0288 - acc: 1.0000 - val_loss: 0.5952 - val_acc:

390/390 [==============================] - 6s - loss: 0.0345 - acc: 0.9974 - val_loss: 0.5458 - val_acc: 0.8465
Epoch 1793/2000
390/390 [==============================] - 6s - loss: 0.0299 - acc: 0.9974 - val_loss: 0.5333 - val_acc: 0.8568
Epoch 1794/2000
390/390 [==============================] - 6s - loss: 0.0413 - acc: 0.9949 - val_loss: 0.5528 - val_acc: 0.8465
Epoch 1795/2000
390/390 [==============================] - 6s - loss: 0.0402 - acc: 0.9974 - val_loss: 0.6452 - val_acc: 0.8338
Epoch 1796/2000
390/390 [==============================] - 6s - loss: 0.0398 - acc: 0.9923 - val_loss: 0.5959 - val_acc: 0.8363
Epoch 1797/2000
390/390 [==============================] - 6s - loss: 0.0367 - acc: 0.9949 - val_loss: 0.5360 - val_acc: 0.8414
Epoch 1798/2000
390/390 [==============================] - 6s - loss: 0.0371 - acc: 0.9923 - val_loss: 0.7214 - val_acc: 0.8082
Epoch 1799/2000
390/390 [==============================] - 6s - loss: 0.0259 - acc: 1.0000 - val_loss: 0.5235 - val_acc:

390/390 [==============================] - 6s - loss: 0.0224 - acc: 1.0000 - val_loss: 0.5078 - val_acc: 0.8465
Epoch 1857/2000
390/390 [==============================] - 6s - loss: 0.0271 - acc: 0.9923 - val_loss: 0.5374 - val_acc: 0.8568
Epoch 1858/2000
390/390 [==============================] - 6s - loss: 0.0287 - acc: 0.9923 - val_loss: 0.4999 - val_acc: 0.8465
Epoch 1859/2000
390/390 [==============================] - 6s - loss: 0.0312 - acc: 0.9949 - val_loss: 0.5148 - val_acc: 0.8363
Epoch 1860/2000
390/390 [==============================] - 6s - loss: 0.0245 - acc: 1.0000 - val_loss: 0.5390 - val_acc: 0.8440
Epoch 1861/2000
390/390 [==============================] - 6s - loss: 0.0253 - acc: 0.9949 - val_loss: 0.5262 - val_acc: 0.8389
Epoch 1862/2000
390/390 [==============================] - 6s - loss: 0.0276 - acc: 1.0000 - val_loss: 0.5092 - val_acc: 0.8465
Epoch 1863/2000
390/390 [==============================] - 6s - loss: 0.0282 - acc: 0.9923 - val_loss: 0.6791 - val_acc:

390/390 [==============================] - 6s - loss: 0.0241 - acc: 1.0000 - val_loss: 0.6139 - val_acc: 0.8338
Epoch 1921/2000
390/390 [==============================] - 6s - loss: 0.0291 - acc: 0.9949 - val_loss: 0.5683 - val_acc: 0.8338
Epoch 1922/2000
390/390 [==============================] - 6s - loss: 0.0225 - acc: 1.0000 - val_loss: 0.5828 - val_acc: 0.8414
Epoch 1923/2000
390/390 [==============================] - 6s - loss: 0.0238 - acc: 0.9974 - val_loss: 0.6046 - val_acc: 0.8363
Epoch 1924/2000
390/390 [==============================] - 6s - loss: 0.0216 - acc: 0.9974 - val_loss: 0.6136 - val_acc: 0.8338
Epoch 1925/2000
390/390 [==============================] - 6s - loss: 0.0357 - acc: 0.9974 - val_loss: 0.5741 - val_acc: 0.8465
Epoch 1926/2000
390/390 [==============================] - 6s - loss: 0.0255 - acc: 1.0000 - val_loss: 0.5484 - val_acc: 0.8568
Epoch 1927/2000
390/390 [==============================] - 6s - loss: 0.0286 - acc: 0.9949 - val_loss: 0.5866 - val_acc:

390/390 [==============================] - 6s - loss: 0.0220 - acc: 0.9974 - val_loss: 0.5388 - val_acc: 0.8491
Epoch 1985/2000
390/390 [==============================] - 6s - loss: 0.0202 - acc: 1.0000 - val_loss: 0.5509 - val_acc: 0.8363
Epoch 1986/2000
390/390 [==============================] - 6s - loss: 0.0272 - acc: 0.9923 - val_loss: 0.5622 - val_acc: 0.8440
Epoch 1987/2000
390/390 [==============================] - 6s - loss: 0.0303 - acc: 0.9923 - val_loss: 0.5559 - val_acc: 0.8414
Epoch 1988/2000
390/390 [==============================] - 6s - loss: 0.0219 - acc: 0.9974 - val_loss: 0.5225 - val_acc: 0.8491
Epoch 1989/2000
390/390 [==============================] - 6s - loss: 0.0196 - acc: 1.0000 - val_loss: 0.5198 - val_acc: 0.8465
Epoch 1990/2000
390/390 [==============================] - 6s - loss: 0.0147 - acc: 1.0000 - val_loss: 0.5095 - val_acc: 0.8414
Epoch 1991/2000
390/390 [==============================] - 6s - loss: 0.0206 - acc: 1.0000 - val_loss: 0.5474 - val_acc:

In [20]:
from keras.models import load_model
model.save('model/model_FCN.h5')  # creates a HDF5 file 'my_model.h5'

In [16]:
classes = model.predict(x_test)

In [17]:
print('classes.shape: ',classes.shape)
np.argmax(classes, axis=1)

classes.shape:  (391, 37)


array([15, 35, 27, 21, 15, 36, 23,  1, 36, 13, 23, 13, 24, 17, 21, 36, 17,
        7, 27, 36, 17, 36, 10, 32, 27,  2,  1, 36, 31, 31, 34, 15,  7,  7,
       35, 34, 36, 10,  6, 10, 11, 11,  7, 34, 11, 30, 21, 36, 24, 15, 13,
       30, 10, 35, 36, 11, 24, 18,  7,  9, 18, 16, 10, 27,  1, 10, 23, 17,
       21, 11,  2, 11, 35,  1, 11, 10, 22, 34, 22,  1, 21, 34,  7,  9,  1,
        9, 21,  3, 29, 36, 36,  2, 17, 10, 24, 36,  5,  7, 36, 35,  5, 36,
       10, 23, 16, 36, 21, 36, 10,  7, 27, 36, 36, 10, 10, 30, 34,  2, 34,
        2, 18,  5, 27, 21, 18, 36, 34,  2, 36, 13, 22, 24, 27,  8, 10, 10,
       22, 30, 17, 35, 18, 10, 32,  7, 35, 15, 32, 36, 34, 16,  7, 27, 13,
       10, 18, 34, 15, 13, 36, 24, 34,  4, 16, 18, 18, 36, 30, 34,  3, 34,
       27, 36, 10,  2,  2, 10, 18,  1, 17, 35, 21,  2,  7,  2, 36,  3, 36,
        7, 10, 32, 34, 36, 33, 36, 24, 36, 36, 35, 36, 34, 23, 35,  7, 10,
       32,  7, 27, 11, 23,  2, 22, 34, 16, 20, 11, 36, 10, 16,  9,  7,  9,
        7, 10, 27, 36, 36

In [5]:
y_test.shape

(391,)

In [18]:
from keras.models import load_model

model.save('model_FCN_ep2000.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')